In [1]:
import yfinance as yf
import pandas as pd
import warnings
import numpy as np
from typing import List
warnings.filterwarnings("ignore")

In [ ]:
test_output = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL','^GSPC'], start="2015-01-01", end="2020-02-21")["Close"].pct_change().dropna()
test_output.head()

In [ ]:
test_output.pct_change()

In [2]:

class TickerAnalytics:
    """A class to calculate and tabulate financial metrics for a list of tickers."""
    
    def __init__(self, tickers: List[str], start: str, end: str, benchmark_ticker: str = '^GSPC', rf_rate: float = 0.0,  interval: str = '1d'):
        """
        Initialize TickerAnalytics with a list of tickers, start and end dates, a benchmark ticker, risk-free rate, and data interval.
        
        Args:
            tickers (List[str]): List of ticker symbols.
            start (str): Start date in 'YYYY-MM-DD' format.
            end (str): End date in 'YYYY-MM-DD' format.
            benchmark_ticker (str): Ticker symbol of the benchmark. Defaults to '^GSPC'.
            rf_rate (float): Risk-free rate. Defaults to 0.0.
            interval (str): Data interval. Defaults to '1d'.
        """
        self.tickers = tickers
        self.benchmark_ticker = benchmark_ticker
        self.rf_rate = rf_rate
        self.data = self.get_data(start, end, interval)
        self.returns = self.calculate_returns()
        self.returns_without_compound = self.calculate_returns_without_compound()
        
    def get_data(self, start: str, end: str, interval: str) -> pd.DataFrame:
        """
        Download historical data for the tickers and benchmark (SP500).
        
        Args:
            start (str): Start date in 'YYYY-MM-DD' format.
            end (str): End date in 'YYYY-MM-DD' format.
            interval (str): Data interval.
            
        Returns:
            pd.DataFrame: DataFrame of historical data.
        """
        data = yf.download(self.tickers+[self.benchmark_ticker], start=start, end=end, interval=interval)["Close"]
        return data
    
    def calculate_returns(self) -> pd.DataFrame:
        """
        Calculate returns for the tickers and benchmark.
        pct_change() is used to calculate the percentage change between the current and a prior element.
        
        Returns:
            pd.DataFrame: DataFrame of returns.
        """
        return self.data.pct_change().dropna()+1
    
    def calculate_returns_without_compound(self) -> pd.DataFrame:
        """
        Calculate returns for the tickers and benchmark without compounding.
        pct_change() is used to calculate the percentage change between the current and a prior element.
        
        Returns:
            pd.DataFrame: DataFrame of returns.
        """
        return self.data.pct_change().dropna()
    
    def calculate_beta(self, ticker: str) -> float:
        """
        Calculate beta for a ticker.
        the covariance of the stock with the market divided by the variance of the market
        the 0,1 index is the covariance of the stock with the market
        the 1,1 index is the variance of the market
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Beta of the ticker.
        """
        cov = np.cov(self.returns[ticker], self.returns[self.benchmark_ticker])
        return cov[0][1]/cov[1][1]
    
    def calculate_alpha(self, ticker: str) -> float:
        """
        Calculate alpha for a ticker.
        the formula is the average return of the stock - the risk free rate - beta * (average return of the market - risk free rate)
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Alpha of the ticker.
        """
        beta = self.calculate_beta(ticker)
        market_return = np.prod(self.returns[self.benchmark_ticker])**(252/len(self.returns[self.benchmark_ticker])) - 1
        individual_ticker_return = np.prod(self.returns[ticker])**(252/len(self.returns[ticker])) - 1
        return individual_ticker_return - (self.rf_rate + beta * (market_return - self.rf_rate))
    
    
    def calculate_alpha_without_compound(self, ticker: str) -> float:
        """
        Calculate the alpha of a given ticker without compounding.
        
        Args:
            ticker (str): The ticker symbol for which to calculate the alpha.
            
        Returns:
            float: The alpha of the ticker.
        """
        beta = self.calculate_beta(ticker)
        market_return = self.returns_without_compound[self.benchmark_ticker].mean()*252
        individual_ticker_return = self.returns_without_compound[ticker].mean()*252
        return individual_ticker_return - (self.rf_rate + beta * (market_return - self.rf_rate))
    

    
    def calculate_sharpe_ratio(self, ticker: str) -> float:
        """
        Calculate Sharpe ratio for a ticker.
        the formula is the average return of the stock - the risk free rate / the standard deviation of the stock
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Sharpe ratio of the ticker.
        """
        individual_ticker_return = np.prod(self.returns[ticker])**(252/len(self.returns[ticker])) - 1
        individual_tick_std = self.returns[ticker].std() * np.sqrt(252)
        return (individual_ticker_return - self.rf_rate)/individual_tick_std
    
    def calculate_sharpe_ratio_without_compound(self, ticker: str) -> float:
        """
        Calculate the Sharpe ratio of a given ticker without compounding.
        
        Args:
            ticker (str): The ticker symbol for which to calculate the Sharpe ratio.
            
        Returns:
            float: The Sharpe ratio of the ticker.
        """
        individual_ticker_return = self.returns_without_compound[ticker].mean() * 252
        individual_tick_std = self.returns_without_compound[ticker].std() * np.sqrt(252)
        return (individual_ticker_return - self.rf_rate)/individual_tick_std
    

    
    def tabulate_metrics(self) -> pd.DataFrame:
        """
        Tabulate various financial metrics for each ticker.
        
        This function calculates and tabulates the following metrics for each ticker:
        - Return (with and without compound)
        - Standard Deviation
        - Covariance with Market
        - Beta
        - Alpha (with and without compound)
        - Sharpe Ratio (with and without compound)
        
        Returns:
            pd.DataFrame: A DataFrame where each row represents a ticker and each column represents a financial metric.
        """
        metrics = pd.DataFrame(index=self.tickers)
        for ticker in self.tickers:
            metrics.loc[ticker, "Return (with compound)"] = np.prod(self.returns[ticker])**(252/len(self.returns[ticker])) - 1
            metrics.loc[ticker, "Return (without compound)"] = self.returns_without_compound[ticker].mean() * 252
            metrics.loc[ticker, "Standard Deviation"] = self.returns[ticker].std() * np.sqrt(252)
            metrics.loc[ticker, "Covariance with Market"] = np.cov(self.returns[ticker], self.returns[self.benchmark_ticker])[0][1]
            metrics.loc[ticker, "Beta"] = self.calculate_beta(ticker)
            metrics.loc[ticker, "Alpha (with compound)"] = self.calculate_alpha(ticker)
            metrics.loc[ticker, "Alpha (without compound)"] = self.calculate_alpha_without_compound(ticker)
            metrics.loc[ticker, "Sharpe Ratio (with compound)"] = self.calculate_sharpe_ratio(ticker)
            metrics.loc[ticker, "Sharpe Ratio (without compound)"] = self.calculate_sharpe_ratio_without_compound(ticker)
        return metrics
        


In [3]:
ticker_analytics = TickerAnalytics(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL'], start="2023-01-30", end="2024-01-31")

[*********************100%%**********************]  6 of 6 completed


In [4]:
ticker_analytics.tabulate_metrics()

,Return (with compound),Return (without compound),Standard Deviation,Covariance with Market,Beta,Alpha (with compound),Alpha (without compound),Sharpe Ratio (with compound),Sharpe Ratio (without compound)
META,1.731254,1.078851,0.391911,0.000115,1.835942,1.314878,0.689047,4.417470,2.752797
AAPL,0.316400,0.295022,0.200334,0.000070,1.124535,0.061365,0.056262,1.579360,1.472646
AMZN,0.584192,0.510485,0.317217,0.000095,1.517918,0.239941,0.188203,1.841618,1.609261
NFLX,0.596943,0.537159,0.376003,0.000078,1.254113,0.312520,0.270888,1.587603,1.428604
GOOGL,0.565190,0.491414,0.293309,0.000087,1.398592,0.248001,0.194467,1.926944,1.675413


In [6]:
print(f"The best alpha is {ticker_analytics.tabulate_metrics()['Alpha (without compound)'].idxmax()} with a value of {ticker_analytics.tabulate_metrics()['Alpha (without compound)'].max()}")

The best alpha is META with a value of 0.6890471876992336


In [7]:
print(f"The best alpha is {ticker_analytics.tabulate_metrics()['Alpha (with compound)'].idxmax()} with a value of {ticker_analytics.tabulate_metrics()['Alpha (with compound)'].max()}")

The best alpha is META with a value of 1.314878160226225


In [9]:
print(f"The best risk weighted return is {ticker_analytics.tabulate_metrics()['Sharpe Ratio (without compound)'].idxmax()} with a value of {ticker_analytics.tabulate_metrics()['Sharpe Ratio (without compound)'].max()}")

The best risk weighted return is META with a value of 2.7527971449744126


In [10]:
print(f"The best risk weighted return is {ticker_analytics.tabulate_metrics()['Sharpe Ratio (with compound)'].idxmax()} with a value of {ticker_analytics.tabulate_metrics()['Sharpe Ratio (with compound)'].max()}")

The best risk weighted return is META with a value of 4.417470162583204


In [8]:
print(f"The weighted average beta is {ticker_analytics.tabulate_metrics()['Beta'].mean()}")

The weighted average beta is 1.4262199465868632


In [5]:
ticker_analytics.returns.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,^GSPC
Ticker,,,,,,
AAPL,1.000000,0.426002,0.516306,0.533325,0.395618,0.704159
AMZN,0.426002,1.000000,0.596966,0.594376,0.361140,0.600268
GOOGL,0.516306,0.596966,1.000000,0.609488,0.321342,0.598162
META,0.533325,0.594376,0.609488,1.000000,0.315019,0.587658
NFLX,0.395618,0.361140,0.321342,0.315019,1.000000,0.418407
^GSPC,0.704159,0.600268,0.598162,0.587658,0.418407,1.000000
